### Prepare data

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet  -P ./data/raw/yellow/ 

In [ ]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

df_yellow = spark.read \
    .parquet('./data/raw/yellow/')

In [ ]:
df_yellow.show(5)

In [ ]:
df_yellow.printSchema()

### HW questions

In [ ]:
# Q1
spark.version

In [ ]:
#Q2
df_yellow = df_yellow.repartition(4)
df_yellow.write.parquet("./data/parquet/yellow/")

In [ ]:
!ls -lh ./data/parquet/yellow/

In [ ]:
#Q3
from pyspark.sql import functions as F

(
    df_yellow.filter(
        F.to_date(F.col("tpep_pickup_datetime")) == "2024-10-15")
    # .orderBy("tpep_pickup_datetime", ascending=False)
).count()

In [ ]:
#Q3 - pandas
from datetime import datetime

import numpy as np
import pandas as pd

df_pd = pd.read_parquet("./data/parquet/yellow/")
df_pd["pickup_date"] = df_pd["tpep_pickup_datetime"].dt.date
df_pd[df_pd["pickup_date"] == datetime.strptime("2024-10-15", "%Y-%m-%d").date()]


In [ ]:
df_pd["tpep_pickup_str"] = df_pd["tpep_pickup_datetime"].astype(str).str.split(" ").str[0]
df_pd["pickup_str"] = df_pd["pickup_date"].astype(str)
df_pd[df_pd["tpep_pickup_str"] != df_pd["pickup_str"]]

In [ ]:
#Q4
(
    df_yellow
        .withColumn("trip_duration", 
                    (F.to_timestamp(F.col("tpep_dropoff_datetime")).cast("long") 
                     - F.to_timestamp(F.col("tpep_pickup_datetime")).cast("long")) / 3600
                )
        .select("tpep_dropoff_datetime", "tpep_pickup_datetime", "trip_duration")
        .orderBy("trip_duration", ascending=False)
).show(3)

In [ ]:
max((pd.to_datetime(df_pd["tpep_dropoff_datetime"])
 - pd.to_datetime(df_pd["tpep_pickup_datetime"])) / np.timedelta64(1, 'h'))

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv  -P ./data/raw/lookup/ 

In [ ]:
df_lookup = spark.read \
        .option("header", "true") \
        .csv("./data/raw/lookup/taxi_zone_lookup.csv")

df_lookup.show()

In [ ]:
(
    df_yellow
    .join(df_lookup, df_yellow.PULocationID == df_lookup.LocationID, "inner")
    .groupBy("Zone").count()
    .orderBy("count")
).head()